In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# RNN を使ったテキスト分類

In [2]:
# !pip install -q tf-nightly
# import tensorflow_datasets as tfds
# !pip3 list | grep tensorflow
# import tensorflow as tf

# !pip3 install sentencepiece
import sentencepiece as spm

import json
import os
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GPUを使うために必要
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# import matplotlib.pyplot as plt

# def plot_graphs(history, metric):
#   plt.plot(history.history[metric])
#   plt.plot(history.history['val_'+metric], '')
#   plt.xlabel("Epochs")
#   plt.ylabel(metric)
#   plt.legend([metric, 'val_'+metric])
#   plt.show()

## Data


### load

In [4]:
sbj_names = [
    ['Eigo']
#     ['Suugaku']
    # ['Eigo', 'Suugaku']
][0]
elmnt_name = 'question'
attr_name = 'knowledge_type'  # 'answer_type'   #
is_remove_xml = True #@param {type:"boolean"}

df = None
for sbj in sbj_names:
    print(sbj)
    attr_csv_path = f'../{sbj}_{attr_name}_ds.tsv'
    df_tmp = pd.read_csv(attr_csv_path, delimiter='\t')
    df = pd.concat([df, df_tmp])

df = df.reset_index(drop=True)
df = df.dropna()  # nan を削除
df

Eigo


,Unnamed: 0,knowledge_type,<instruction/>,contents
0,0,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問１</label> <ansColumn id=""A1"">1</ansC..."
1,1,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問２</label> <ansColumn id=""A2"">2</ansC..."
2,2,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問３</label> <ansColumn id=""A3"">3</ansC..."
3,3,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問４</label> <ansColumn id=""A4"">4</ansC..."
4,4,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,"<label>問５</label> <ansColumn id=""A5"">5</ansC..."
...,...,...,...,...
2630,2630,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問２</label> <data id=""D44"" type=""text..."
2631,2631,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問３</label> <data id=""D45"" type=""text..."
2632,2632,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問４</label> <data id=""D46"" type=""text..."
2633,2633,Other,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"<label>問５</label> <data id=""D47"" type=""text..."


In [5]:
## XMLタグ除去
if is_remove_xml:
    import re
    def remove_xml(xml_child_str):
        xml_child_str = re.sub('<.*?>', '', xml_child_str)
        return re.sub('</.*?>', '', xml_child_str)

    # test_idx = 200
    # print(df['contents'][test_idx])
    # print(remove_xml(df['contents'][test_idx]))
    df['contents'] = df['contents'].map(remove_xml)
df

,Unnamed: 0,knowledge_type,<instruction/>,contents
0,0,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,問１ 1 ① circumstance ② pursue ③ universal...
1,1,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,問２ 2 ① definite ② delicate ③ recently ...
2,2,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,問３ 3 ① patience ② magic ③ manager ④ ba...
3,3,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,問４ 4 ① conference ② confidence ③ contine...
4,4,PRN,次の問い(問１～５)のそれぞれの単語①～④のうちから，アクセント(第一強勢)のある母音の発音...,問５ 5 ① photograph ② profit ③ notice ④ ...
...,...,...,...,...
2630,2630,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"問２ According to paragraph (3) , what did p..."
2631,2631,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"問３ According to paragraph (4) , which opin..."
2632,2632,R_QA,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,"問４ According to paragraph (6) , what is on..."
2633,2633,Other,次の問い（問１～５）の 47 ～ 51 に入れるのに最も適当なものを，それぞれ下の①～④のう...,問５ What would be the best title for this pa...


### Tokenize
SentencePiece を使用。
- タグ あり／なし

In [6]:
# df_tmp = df[['<instruction/>', 'contents']]
# df_tmp['<instruction/>'][6]

In [7]:
m_dir = '_logs/SentencePiece'
os.makedirs(m_dir, exist_ok=True)
df.to_csv(f'{m_dir}/tmp.txt', sep='\t')

# arg_str = '--input={m_dir}/tmp.txt --model_prefix={m_dir}/m_user ' + '--user_defined_symbols=<sep>,<cls>' + ',<ansColumn/>,<label>' + ' --vocab_size=2000'
# spm.SentencePieceTrainer.train(arg_str)

spm.SentencePieceTrainer.train(f'--input={m_dir}/tmp.txt --model_prefix={m_dir}/m  --user_defined_symbols=<sep>,<cls>,<pad>   --vocab_size=2000')
sp = spm.SentencePieceProcessor()  # model_file='SentencePiece/test_model.model'

sp.load(f'{m_dir}/m.model')

True

In [8]:
# # encode: text => id
# tokenized_tokens =  sp.encode_as_pieces('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
# print(tokenized_tokens)

# tokenized_ids = sp.encode_as_ids('次の問い(問１～３)の会話の 17 ～ 19 に入れるのに最も適当なものを，それぞれ以下の①～④のうちから一つずつ選べ。	')
# print(tokenized_ids)

# decoded_text = sp.decode(tokenized_ids)
# print(decoded_text)

In [9]:
# example_content = df_tmp['contents'][20]
# print(example_content, sp.encode_as_pieces(example_content))

In [10]:
# for index in encoded_string:
#   print('{} ----> {}'.format(index, encoder.decode([index])))

## Train 用データの準備

In [11]:
word2index = {}
# 系列を揃えるためのパディング文字列<pad>を追加
# パディング文字列のIDは0とする
word2index.update({"<pad>":0})

for inst, cont in zip(df['<instruction/>'], df['contents']):
#     try:
    tokens = sp.encode_as_pieces(inst + cont)
    for word in tokens:
            if word in word2index: continue
            word2index[word] = len(word2index)
#     except TypeError:
#         print(f'[Error] <instruction/> が nan です。')
#         print(f'    inst : {inst}')
#         print(f'    cont : {cont}')

print("vocab size : ", len(word2index))


vocab size :  2100


In [12]:
## set_dict から自動抽出する！
# attr_name = 'knowledge_type'  # 'answer_type'

categories = set()
for sbj in sbj_names:
    with open(f'../class_set/{sbj}-{elmnt_name}-{attr_name}.json') as f:
        categories |= set(json.load(f))   # sbj_names = ['Eigo', ]

# print(categories)

# categories = [
#     'sentence', 
#     'term_person', 'term_location', 'term_time', 'term_other',
#     'referenceSymbol',
#     'image_graph', 'image_photo', 'image_map', 'image_table', 'image_other',
#     'formula', 
#     'orthography',
#     'other',
#     # 組み合わせ系（仮追加）
#     '(symbol-sentence)*2', '(symbol-sentence)*3', '(symbol-sentence)*4', '(symbol-term_location)*3', '(symbol-term_other)*3', '(symbol-term_other)*3',
#     '(symbol-symbol)*4',
#     '(term_location-term_location-term_location)', 'term_location-term_location-term_location-term_location',
#     '(term_location-term_location-term_location)', '(term_location-term_location-term_location)',
#     '(term_other-term_other-term_other)',
#     'term_other-term_other-term_other',
#     'sentence-sentence',
#     'sentence-sentence-sentence',
#     'symbol-symbol',
#     'symbol-symbol-symbol',
#     'symbol-symbol-symbol-symbol',
#     'o(symbol-symbol-symbol-symbol)',
#     'o(symbol-symbol-symbol)',
# ]

categories = list(categories)
categories.sort()    # 入れないと、クラス番号が変わってしまい、再現実験ができないので注意？
print(categories)
print(len(categories))

['DIC_O', 'DIC_O, DIS_S, R_ENT', 'DIC_O, DIS_W', 'DIC_O, DIS_W, GK', 'DIC_O, EG', 'DIC_O, EG, DIS_W', 'DIC_O, EG, GK', 'DIC_O, GK', 'DIC_O, Other', 'DIC_O, SEL', 'DIC_O,DIS_W', 'DIC_O,GK', 'DIS_C, GK', 'DIS_O', 'DIS_O, DIS_W', 'DIS_O, DIS_W, GK', 'DIS_O, GK', 'DIS_O, Other', 'DIS_S', 'DIS_S, IC_O', 'DIS_S, R_ENT', 'DIS_S, R_ENT, IC_G', 'DIS_S, R_ENT, IC_M', 'DIS_S, R_ENT, IC_O', 'DIS_S, R_ENT, IC_T', 'DIS_S, R_ENT, R_SUM', 'DIS_S,R_ENT', 'DIS_S,R_ENT,R_SUM', 'DIS_S,R_QA', 'DIS_W', 'DIS_W, GK', 'DIS_W, R_ENT', 'DIS_W, R_ENT, IC_G', 'DIS_W, R_ENT, IC_G, IC_P', 'DIS_W, R_ENT, IC_M', 'DIS_W, R_ENT, IC_O', 'DIS_W, R_ENT, IC_O, IC_P', 'DIS_W, R_ENT, IC_P', 'DIS_W, R_ENT, IC_P, IC_T', 'DIS_W, R_ENT, IC_T', 'DIS_W, R_ENT, R_SUM', 'DIS_W, R_ENT, R_SUM, IC_O', 'DIS_W, R_QA', 'DIS_W,DIC_O', 'DIS_W,EG', 'DIS_W,GK', 'DIS_W,IC_P', 'DIS_W,R_ENT', 'DIS_W,R_ENT,IC_P', 'DIS_W,R_QA', 'DIS_W,R_SUM', 'DIS_W,R_SUM,IC_T', 'EG', 'EG, ', 'EG, DIS_S', 'EG, DIS_W', 'EG, DIS_W, GK', 'EG, GK', 'EG,DIS_W', 'EG,SEL'

In [13]:
## 系列の長さを揃えてバッチでまとめる

from sklearn.model_selection import train_test_split
import random
from sklearn.utils import shuffle

cat2index = {}
for cat in categories:
    if cat in cat2index: continue
    cat2index[cat] = len(cat2index)

def sentence2index(sentence):
    tokens = sp.encode_as_pieces(sentence)
    # print(tokens)
    return [word2index[w] for w in tokens]

def category2index(cat):
    return [cat2index[cat]]

index_datasets_c_xml_tmp = []
index_datasets_category = []

# 系列の長さの最大値を取得。この長さに他の系列の長さをあわせる
max_len = 0
for inst, cont, category in zip(df['<instruction/>'], df['contents'], df[attr_name]):
    index_c_xml = sentence2index(inst + cont)
    index_category = category2index(category)
    index_datasets_c_xml_tmp.append(index_c_xml)
    index_datasets_category.append(index_category)
    if max_len < len(index_c_xml):
        max_len = len(index_c_xml)

# 系列の長さを揃えるために短い系列にパディングを追加
# 後ろパディングだと正しく学習できなかったので、前パディング
index_datasets_c_xml = []
for c_xml in index_datasets_c_xml_tmp:
    for i in range(max_len - len(c_xml)):
        c_xml.insert(0, 0) # 前パディング
#     c_xml.append(0)　# 後ろパディング
    index_datasets_c_xml.append(c_xml)

train_x, test_x, train_y, test_y = train_test_split(index_datasets_c_xml, index_datasets_category, train_size=0.7)

# データをバッチでまとめるための関数
def train2batch(c_xml, category, batch_size=100):
    c_xml_batch = []
    category_batch = []
    c_xml_shuffle, category_shuffle = shuffle(c_xml, category)
    for i in range(0, len(c_xml), batch_size):
        c_xml_batch.append(c_xml_shuffle[i:i+batch_size])
        category_batch.append(category_shuffle[i:i+batch_size])
    return c_xml_batch, category_batch

## モデルの作成

In [14]:
class LSTMClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        # <pad>の単語IDが0なので、padding_idx=0としている
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        # batch_first=Trueが大事！
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        #embeds.size() = (batch_size × len(sentence) × embedding_dim)
        _, lstm_out = self.lstm(embeds)
        # lstm_out[0].size() = (1 × batch_size × hidden_dim)
        tag_space = self.hidden2tag(lstm_out[0])
        # tag_space.size() = (1 × batch_size × tagset_size)

        # (batch_size × tagset_size)にするためにsqueeze()する
        tag_scores = self.softmax(tag_space.squeeze())
        # tag_scores.size() = (batch_size × tagset_size)

        return tag_scores  

    def load_weights(self, load_m_path='_logs/test/LSTM_classifier.pth',):
        if load_m_path is not None:
            param = torch.load(load_m_path)
            self.load_state_dict(param)
            print(f'[info] {load_m_path} loaded !')

    def save(self, save_f_path='_logs/test/LSTM_classifier.pth',):
        torch.save(self.state_dict(), save_f_path)

    
# 単語の埋め込み次元数上げた。精度がそこそこアップ！ハイパーパラメータのチューニング大事。
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
VOCAB_SIZE = len(word2index)
TAG_SIZE = len(categories)

# model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE).to(device)

# loss_function = nn.NLLLoss()
# # SGDからAdamに変更。特に意味はなし
# optimizer = optim.Adam(model.parameters(), lr=0.001)


---

## Eval

In [15]:
option = ''

# 重み読み込み
best_m_dir = [
#     'best-kwlg_type_sbj_en'
#     'best-kwlg_type_sbj_math'
    'best-kwlg_type_sbj_en_noXML'
    # 'best-kwlg_type_sbj_x2'
][0]
# option = '_500epc'

model = LSTMClassifier(EMBEDDING_DIM, HIDDEN_DIM, VOCAB_SIZE, TAG_SIZE)
model.load_weights(f'_logs/{best_m_dir}/LSTM_classifier{option}.pth')
# to(device)でモデルがGPU対応する
model.to(device)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
test_num = len(test_x)
a = 0
with torch.no_grad():
    title_batch, category_batch = train2batch(test_x, test_y)

    for i in range(len(title_batch)):
        title_tensor = torch.tensor(title_batch[i], device=device)
        category_tensor = torch.tensor(category_batch[i], device=device)

        out = model(title_tensor)
        _, predicts = torch.max(out, 1)
        for j, ans in enumerate(category_tensor):
            if predicts[j].item() == ans.item():
                a += 1
#             else:
                print(predicts[j].item(), ans.item())
print("predict : ", a / test_num)
# predict :  0.6967916854948034

___